# Introduction
This notebook contains the python code used to clean and manipulate  a dataset downloaded from ['Our World in Data'](https://github.com/owid) website using this [link](https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv). Subsets of data were generated and used to construct a SQL database which then utilized to create monthly aggregated dataset of cases that eventually was used to create visualization in [Tableau Software](https://www.tableau.com).


## Importing libraries

In [1]:
# importing required libraries

import pandas as pd
import numpy as np

## Loading the dataset

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")
df.head(2)



,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


## Exploring and cleaning the dataset

In [3]:
df.shape

(252742, 67)

In [4]:
## adding ids identifiers to used in creating SQl database

df.insert(0,"vacc_id", np.arange(1,len(df)+1))
df.insert(0,"case_id", np.arange(1,len(df)+1))

In [5]:
# checking columns data type.
df.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252742 entries, 0 to 252741
Data columns (total 69 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   case_id                                     252742 non-null  int64  
 1   vacc_id                                     252742 non-null  int64  
 2   iso_code                                    252742 non-null  object 
 3   continent                                   238548 non-null  object 
 4   location                                    252742 non-null  object 
 5   date                                        252742 non-null  object 
 6   total_cases                                 238404 non-null  float64
 7   new_cases                                   238142 non-null  float64
 8   new_cases_smoothed                          236938 non-null  float64
 9   total_deaths                                218831 non-null  float64
 

In [6]:
# Saving the modified dataset in local folder
df.to_csv("complete_row_data.csv", index=False)


A subset of data that related to the number of cases and deaths will be saved in a separate file. Vacinnation and demographics related data will stored in different files. This will help removing redundant information before construction SQL database.  

## Creating a dataset contains cases related data

In [7]:
# Selecting the realevant columns and casting to appropriate type of data 
covid_19_cases = pd.read_csv("complete_row_data.csv",
                             usecols = ['case_id','vacc_id', 'iso_code',
                                        'date','new_cases','new_deaths'],
                            dtype={'new_cases': 'Int64', 'new_deaths': 'Int64'} )
covid_19_cases.head()

,case_id,vacc_id,iso_code,date,new_cases,new_deaths
0,1,1,AFG,2020-02-24,5,<NA>
1,2,2,AFG,2020-02-25,0,<NA>
2,3,3,AFG,2020-02-26,0,<NA>
3,4,4,AFG,2020-02-27,0,<NA>
4,5,5,AFG,2020-02-28,0,<NA>


In [8]:
# saving cases dataset
covid_19_cases.to_csv("covid_19_cases.csv", index=False)

## Creating a dataset contains vaccination related data

In [9]:
# Selecting the realevant columns and casting to appropriate type of data 
vacc = pd.read_csv("complete_row_data.csv", 
                      usecols = ['vacc_id','iso_code','total_tests','new_tests', 'positive_rate', 
                                 'people_vaccinated', 'people_fully_vaccinated'], 
                      dtype={'total_tests': 'Int64', 'new_cases': 'Int64', 
                                    'new_tests': 'Int64','people_vaccinated': 'Int64',
                                    'people_fully_vaccinated': 'Int64'})
vacc.head()

,vacc_id,iso_code,total_tests,new_tests,positive_rate,people_vaccinated,people_fully_vaccinated
0,1,AFG,<NA>,<NA>,NaN,<NA>,<NA>
1,2,AFG,<NA>,<NA>,NaN,<NA>,<NA>
2,3,AFG,<NA>,<NA>,NaN,<NA>,<NA>
3,4,AFG,<NA>,<NA>,NaN,<NA>,<NA>
4,5,AFG,<NA>,<NA>,NaN,<NA>,<NA>


In [10]:
vacc.shape 

(252742, 7)

In [11]:
vacc.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252742 entries, 0 to 252741
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   vacc_id                  252742 non-null  int64  
 1   iso_code                 252742 non-null  object 
 2   total_tests              79387 non-null   Int64  
 3   new_tests                75403 non-null   Int64  
 4   positive_rate            95927 non-null   float64
 5   people_vaccinated        68120 non-null   Int64  
 6   people_fully_vaccinated  65459 non-null   Int64  
dtypes: Int64(4), float64(1), int64(1), object(1)
memory usage: 27.1 MB


In [12]:
# Removing rows that contain "NAN" in all columns.
# Threshold was set =3 to prevent keeping rows that contain data in "vacc_id" only.
# This will significantly reduces number of rows and therefore the data size.
vacc = vacc.dropna(axis = 0,thresh =3)
vacc.shape 

(135080, 7)

In [13]:
vacc.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135080 entries, 364 to 252633
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   vacc_id                  135080 non-null  int64  
 1   iso_code                 135080 non-null  object 
 2   total_tests              79387 non-null   Int64  
 3   new_tests                75403 non-null   Int64  
 4   positive_rate            95927 non-null   float64
 5   people_vaccinated        68120 non-null   Int64  
 6   people_fully_vaccinated  65459 non-null   Int64  
dtypes: Int64(4), float64(1), int64(1), object(1)
memory usage: 15.5 MB


In [14]:
# saving vaccination dataset
vacc.to_csv("vaccination.csv", index=False)

## Creating a dataset contains the demoghraphic information

In [15]:
demographic = pd.read_csv("complete_row_data.csv", 
                          usecols = ['iso_code','continent', 'location',
                                     'aged_65_older', 'aged_70_older','diabetes_prevalence', 
                                     'female_smokers','male_smokers','population'],
                          dtype={'population': 'Int64'} )
demographic.head()


,iso_code,continent,location,aged_65_older,aged_70_older,diabetes_prevalence,female_smokers,male_smokers,population
0,AFG,Asia,Afghanistan,2.581,1.337,9.59,NaN,NaN,41128772
1,AFG,Asia,Afghanistan,2.581,1.337,9.59,NaN,NaN,41128772
2,AFG,Asia,Afghanistan,2.581,1.337,9.59,NaN,NaN,41128772
3,AFG,Asia,Afghanistan,2.581,1.337,9.59,NaN,NaN,41128772
4,AFG,Asia,Afghanistan,2.581,1.337,9.59,NaN,NaN,41128772


In [16]:
demographic.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252742 entries, 0 to 252741
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   iso_code             252742 non-null  object 
 1   continent            238548 non-null  object 
 2   location             252742 non-null  object 
 3   aged_65_older        201534 non-null  float64
 4   aged_70_older        202616 non-null  float64
 5   diabetes_prevalence  213738 non-null  float64
 6   female_smokers       153992 non-null  float64
 7   male_smokers         151885 non-null  float64
 8   population           251658 non-null  Int64  
dtypes: Int64(1), float64(5), object(3)
memory usage: 57.3 MB


In [17]:
# removing duplicates recordes
unique_demographic = demographic.drop_duplicates()
# reset the index
unique_demographic.reset_index(drop=True,  inplace=True)
unique_demographic.head()

,iso_code,continent,location,aged_65_older,aged_70_older,diabetes_prevalence,female_smokers,male_smokers,population
0,AFG,Asia,Afghanistan,2.581,1.337,9.59,NaN,NaN,41128772
1,OWID_AFR,NaN,Africa,NaN,NaN,NaN,NaN,NaN,1426736614
2,ALB,Europe,Albania,13.188,8.643,10.08,7.1,51.2,2842318
3,DZA,Africa,Algeria,6.211,3.857,6.73,0.7,30.4,44903228
4,AND,Europe,Andorra,NaN,NaN,7.97,29.0,37.8,79843


In [18]:
unique_demographic.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   iso_code             248 non-null    object 
 1   continent            235 non-null    object 
 2   location             248 non-null    object 
 3   aged_65_older        193 non-null    float64
 4   aged_70_older        195 non-null    float64
 5   diabetes_prevalence  206 non-null    float64
 6   female_smokers       147 non-null    float64
 7   male_smokers         145 non-null    float64
 8   population           247 non-null    Int64  
dtypes: Int64(1), float64(5), object(3)
memory usage: 57.8 KB


In [19]:
# The final dataset has only "57.3 KB" ibstead of "57.8 MB"
# saving demographic dataset
unique_demographic.to_csv("unique_demographic.csv", index=False)